<a href="https://colab.research.google.com/github/Medhansh-Xplores/Code_Snippet_Search_Engine/blob/main/Code_Snippet_Search_Engine_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dataset

In [ ]:
import pandas as pd

df=pd.read_csv("/content/extended_programming_code_snippets.csv")
df.head(5)

,Query,Code_Snippet,Language,Tags
0,How to create a list comprehension in Python?,[x**2 for x in range(10)],Python,tutorial
1,How to handle missing data in pandas?,"df.fillna(0, inplace=True)",Python,example
2,How to use a lambda function in Python?,lambda x: x + 2,Python,advanced
3,How to create a REST API in Flask?,from flask import Flask\napp = Flask(__name__)...,Python,tutorial
4,How to perform matrix multiplication in numpy?,"import numpy as np\nnp.dot(A, B)",Python,advanced


In [ ]:
df['Language'].value_counts()

,count
Language,
SQL,627
Java,618
Shell,601
JavaScript,595
Python,574
C++,5


Converting to lowercase

In [ ]:
df['Query'] = df['Query'].str.lower()
df['Code_Snippet'] = df['Code_Snippet'].str.lower()
df['Tags'] = df['Tags'].str.lower()

Tokenization

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess_data(data):
    inputs = ["generate code: " + query for query in data["Query"]]
    targets = data["Code_Snippet"].tolist()
    input_encodings = tokenizer(inputs, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    target_encodings = tokenizer(targets, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    return input_encodings, target_encodings

input_encodings, target_encodings = preprocess_data(df)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
input_encodings

{'input_ids': tensor([[3806, 1081,   10,  ...,    0,    0,    0],
        [3806, 1081,   10,  ...,    0,    0,    0],
        [3806, 1081,   10,  ...,    0,    0,    0],
        ...,
        [3806, 1081,   10,  ...,    0,    0,    0],
        [3806, 1081,   10,  ...,    0,    0,    0],
        [3806, 1081,   10,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

Dataset Preparation

In [ ]:
import torch

class CodeSnippetDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            "labels": self.targets["input_ids"][idx],
        }

dataset = CodeSnippetDataset(input_encodings, target_encodings)


Fine Tuning Model

In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

training_args = TrainingArguments(
    output_dir="./t5_finetuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=1,
    load_best_model_at_end=True,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
)

trainer.train()

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

...

Epoch,Training Loss,Validation Loss
1,1.074600,0.069046
2,0.079900,0.012499
3,0.045700,0.009058
4,0.026400,0.008608


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,1.074600,0.069046
2,0.079900,0.012499
3,0.045700,0.009058
4,0.026400,0.008608
5,0.023400,0.008519


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=3775, training_loss=0.19883041078681188, metrics={'train_runtime': 28603.0174, 'train_samples_per_second': 0.528, 'train_steps_per_second': 0.132, 'total_flos': 510915300556800.0, 'train_loss': 0.19883041078681188, 'epoch': 5.0})

Saving Fine Tuned Model


In [ ]:
model.save_pretrained("/content/drive/MyDrive/t5_finetuned_model")
tokenizer.save_pretrained("/content/drive/MyDrive/t5_finetuned_model")

('/content/drive/MyDrive/t5_finetuned_model/tokenizer_config.json',
 '/content/drive/MyDrive/t5_finetuned_model/special_tokens_map.json',
 '/content/drive/MyDrive/t5_finetuned_model/spiece.model',
 '/content/drive/MyDrive/t5_finetuned_model/added_tokens.json')

Search Engine

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/t5_finetuned_model")
tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/t5_finetuned_model")

def generate_code(query):
    query = query.lower()
    input_text = "generate code: " + query
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True)
    outputs = model.generate(input_ids, max_length=512, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

while True:
    query = input("\n\nYour Query or type (exit) to leave : \n  ")
    if query == "exit":
        print("\nGood Bye...")
        break
    code = generate_code(query)
    print("\n Code Snipet : \n", code)



Your Query or type (exit) to leave : 
  how to create a list comprehension in python?

 Code Snipet : 
 [x**2 for x in range(10)]


Your Query or type (exit) to leave : 
  how to perform matrix multiplication in numpy?	

 Code Snipet : 
 import numpy as np np.dot(a, b)


Your Query or type (exit) to leave : 
   how to create a rest api in flask?

 Code Snipet : 
 from flask import flask app = flask(__name__) @app.route('/') def home(): return 'hello, api!'


Your Query or type (exit) to leave : 
  how to add two numbers

 Code Snipet : 
 grep 'double digits'


Your Query or type (exit) to leave : 
  how to add two numbers in python

 Code Snipet : 
 [x**2 for x in range(10)]


Your Query or type (exit) to leave : 
  exit

Good Bye...
